In [37]:
import os
import gym
import slimevolleygym
import numpy as np

from stable_baselines.ppo1 import PPO1
from stable_baselines import TRPO
from stable_baselines.common.policies import MlpPolicy
from stable_baselines import logger
from stable_baselines.common.callbacks import EvalCallback
from stable_baselines.bench import Monitor

from model import BnnPolicy

In [44]:
import gym
import slimevolleygym
import tqdm.notebook as tqdm

def evaluate(p1, p2, n=100):
    env = gym.make("SlimeVolley-v0")
    p1_scores=[]
    p2_scores=[]
    
    for i in tqdm.tqdm(range(n)):
        obs1 = env.reset()
        obs2 = obs1 # both sides always see the same initial observation.

        done = False
        total_reward = 0

        while not done:

            action1, _ = p1.predict(obs1)
            action2, _ = p2.predict(obs2)

            obs1, reward, done, info = env.step(action1, action2) # extra argument
            obs2 = info['otherObs']

            total_reward += reward
            # env.render()
        p1_scores.append(total_reward)
        p2_scores.append(-total_reward)
        
    p1_score = np.mean(p1_scores)
    p2_score = np.mean(p2_scores)
    
    p1_var = np.std(p1_scores)
    p2_var = np.std(p2_scores)
    
    print(f"Player 1: {p1_score} +- {p1_var:.3f}")
    print(f"Player 2: {p2_score} +- {p2_var:.3f}")
        
    return p1_score, p2_score, p1_var, p2_var

In [39]:
ppo_dnn = PPO1.load(os.path.join("exp/self/ppo-dnn", "final_model"))
ppo_bnn = PPO1.load(os.path.join("exp/self/ppo-bnn", "final_model"))

Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.


In [33]:
dnn_score, bnn_score, dnn_var, bnn_var = evaluate(dnn, bnn, n=1000)


DNN: -4.031 += 1.279
BNN: 4.031 += 1.279


In [36]:
bnn_score, dnn_score, bnn_var, dnn_var = evaluate(bnn, dnn, n=100)


BNN: 4.27 +- 1.112
DNN: -4.27 +- 1.112


In [45]:
trpo_dnn = TRPO.load(os.path.join("exp/self/trpo-dnn", "final_model"))

Loading a model without an environment, this model cannot be trained until it has a valid environment.


In [47]:
evaluate(trpo_dnn, ppo_dnn, n=100)
evaluate(trpo_dnn, ppo_bnn, n=100)


Player 1: -0.57 +- 2.688
Player 2: 0.57 +- 2.688



Player 1: -4.25 +- 1.081
Player 2: 4.25 +- 1.081


(-4.25, 4.25, 1.0805091392487154, 1.0805091392487154)